<a href="https://colab.research.google.com/github/jjossie/jbeds-road-signs/blob/main/Eriks_Attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Note: After you run this cell, the training and test data will be available in
# the file browser. (Click the folder icon on the left to view it)
#
# If you don't see the data after the cell completes, click the refresh button
# in the file browser (folder icon with circular arrow)

# First, let's download and unzip the data
!echo "Downloading files..."
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/training1.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/training2.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/test.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/test_partial.zip

!echo "Unzipping files..."
!unzip -q /content/training1.zip
!unzip -q /content/training2.zip
!unzip -q /content/test.zip
!unzip -q /content/test_partial.zip

# Combine the two traning directories
!echo "Merging training data..."
!mkdir /content/training
!mv /content/training1/* /content/training
!mv /content/training2/* /content/training

# Cleanup
!echo "Cleaning up..."
!rmdir /content/training1
!rmdir /content/training2
!rm training1.zip
!rm training2.zip
!rm test.zip
!rm test_partial.zip

!echo "Data ready."


Unzipping files...
Merging training data...
Cleaning up...
Data ready.


In [30]:
!pip install keras-resnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow tensorflow-addons

In [73]:
# Import libraries
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.metrics import Precision, Recall
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.applications import InceptionV3
from keras.applications import VGG16

from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import classification_report

In [66]:
# We're using keras' ImageDataGenerator class to load our image data.
# See (https://keras.io/api/preprocessing/image/#imagedatagenerator-class) for details
#
# A couple of things to note:
# 1. We're specifying a number for the seed, so we'll always get the same shuffle and split of our images.
# 2. Class names are inferred automatically from the image subdirectory names.
# 3. We're splitting the training data into 80% training, 20% validation.


training_dir = '/content/training/'

image_size = (100, 100)

# Split up the training data images into training and validation sets
# We'll use and ImageDataGenerator to do the splits
# ImageDataGenerator can also be used to do preprocessing and augmentation on the files as can be seen with rescale

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=image_size,
    subset="training",
    batch_size=32,
    class_mode='categorical',
    seed=42,
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    training_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset="validation",
    seed=42
)


Found 31368 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


In [78]:
# Build a model...
kes_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

inc_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

model = Sequential()
model.add(inc_model)
model.add(Flatten())
model.add(Dense(43, activation='softmax'))  # Replace num_classes with the correct number of classes

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

training_history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        epochs=6,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size
        )

Epoch 1/6
980/980 [==============================] - 126s 118ms/step - loss: 1.7219 - accuracy: 0.5140 - val_loss: 0.5462 - val_accuracy: 0.8426
Epoch 2/6
980/980 [==============================] - 114s 116ms/step - loss: 0.2052 - accuracy: 0.9385 - val_loss: 0.3542 - val_accuracy: 0.9227
Epoch 3/6
980/980 [==============================] - 113s 116ms/step - loss: 0.1115 - accuracy: 0.9669 - val_loss: 0.3731 - val_accuracy: 0.9111
Epoch 4/6
980/980 [==============================] - 111s 113ms/step - loss: 0.0889 - accuracy: 0.9731 - val_loss: 0.2937 - val_accuracy: 0.9366
Epoch 5/6
980/980 [==============================] - 111s 114ms/step - loss: 0.0908 - accuracy: 0.9741 - val_loss: 0.4412 - val_accuracy: 0.8955
Epoch 6/6
980/980 [==============================] - 113s 115ms/step - loss: 0.0730 - accuracy: 0.9798 - val_loss: 0.1978 - val_accuracy: 0.9485


In [79]:
test_dir = '/content/'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        classes=['test'],
        target_size=image_size,
        class_mode='categorical',
        shuffle=False)
probabilities = model.predict(test_generator)

Found 12630 images belonging to 1 classes.
395/395 [==============================] - 15s 38ms/step


In [80]:
predicted_labels = [np.argmax(probas) for probas in probabilities]

true_labels = test_generator.classes

In [81]:
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted', zero_division=0)
recall = recall_score(true_labels, predicted_labels, average='weighted', zero_division=0)
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Accuracy: 0.004117181314330958
Precision: 1.0
Recall: 0.004117181314330958
F1 Score: 0.008200599274562372
